In [3]:
import os
import zipfile
path_to_zip_file = os.getcwd()+'/11-785-s22-hw1p2.zip'
directory_to_extract_to = ''
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

In [1]:
import torch

In [3]:
torch.cuda.is_available()

False

In [4]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

class Network(torch.nn.Module):
    def __init__(self, in_size):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        # in_size = 13
        # layers = [
        #     nn.Linear(in_size, 512),
        #     nn.BatchNorm1d(512),
        #     nn.ReLU(),
        #     nn.Linear(512, 256),
        #     nn.BatchNorm1d(256),
        #     nn.ReLU(),
        #     nn.Linear(256, 40)
        # ] # 72%
        layers = [
            nn.Linear(in_size, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 40)
        ] 
        #79% with [512,256,128] with relu, BN 
        #82% [1024,512,256,256,128] leakyRL,BN,
        #   [1024,512,512,256,256,128] leakyRL, BN, 2DO
        self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.laysers(A0)
        return x

ModuleNotFoundError: No module named 'torch'

In [ ]:
def my_csv_reader(csv_file, col = 0):
    file_content = []
    with open(csv_file) as f:
        f_csv = csv.reader(f)
        for irow,row in enumerate(f_csv):
            if col == 'all':
                file_content.append(row)
            else:
                file_content.append(row[col])
    return file_content
    
class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None, test_order_csv = None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.X_names = os.listdir(self.X_dir)
        if partition == 'test-clean':
            #get test order as X_names
            
            self.X_names = my_csv_reader(test_order_csv)[1:]

        try:
            self.Y_dir = data_path + "/" + partition +"/transcript/"
            self.Y_names = os.listdir(self.Y_dir)
            assert(len(self.X_names) == len(self.Y_names))
        except:
            self.Y_names = None

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
        
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)

            if self.Y_names:
                Y_path = self.Y_dir + self.Y_names[j]
                label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]
            else:
                label = [self.PHONEMES.index('SIL') for i in range(len(X_data))]
            Y.append(np.array(label))
            
            
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            self.X, self.Y = X, Y
        else:
            self.X = np.pad(X, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)
            self.Y = Y
            # TODO: self.X, self.Y = ... 
             
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            xx = self.X[i:i+2*self.context+1].flatten()
            yy = self.Y[i]
        return xx, yy
    



In [ ]:


def train(args, model, device, train_samples, optimizer, criterion, epoch):
    model.train()
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, dev_samples):
    model.eval()
    true_y_list = []
    pred_y_list = []
    total_loss = 0
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                loss = criterion(output, true_y) 
                total_loss = total_loss + loss.item()
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    ave_loss = total_loss/len(test_loader) #len(test_loader) is the #of batches
    return train_accuracy, ave_loss


# def main(args):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
#     model = Network().to(device)
#     optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    
#     criterion = torch.nn.CrossEntropyLoss()
#     # If you want to use full Dataset, please pass None to csvpath
#     #train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath="/content/train_filenames_subset_8192_v2.csv")
#     train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100")
#     dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")

#     for epoch in range(1, args['epoch'] + 1):
#         train(args, model, device, train_samples, optimizer, criterion, epoch)
#         test_acc, test_loss = test(args, model, device, dev_samples)
#         print('Dev accuracy ', test_acc, ' Dev loss ', test_loss)
         

In [ ]:
args = {
        'batch_size': 512,
        'context': 20,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 0.001,
        'epoch': 20
    }
#main(args)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

in_size =  (2 * args['context'] + 1)*13
model = Network(in_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=args['lr'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3)
criterion = torch.nn.CrossEntropyLoss()
# If you want to use full Dataset, please pass None to csvpath
#train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath="/content/train_filenames_subset_8192_v2.csv")
train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100")
dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")

for epoch in range(1, args['epoch'] + 1):
    train(args, model, device, train_samples, optimizer, criterion, epoch)
    test_acc, test_loss = test(args, model, device, dev_samples)
    print('Dev accuracy ', test_acc, ', test loss', test_loss)
    scheduler.step(test_loss)
    #save model
    output_file_name = 'h1p2_model_epoch%d' % epoch + '.pth'
    torch.save(model.state_dict(), output_file_name)
    print(output_file_name)

In [ ]:
#test
test_samples = LibriSamples(data_path = args['LIBRI_PATH'], sample=20000, shuffle=False, partition="test-clean", test_order_csv="/content/test_order.csv")
print(test_samples.X_names)

In [ ]:
model.eval()
pred_y_list = []
with torch.no_grad():
    for i in range(len(test_samples)):
        X, Y = test_samples[i]

        test_items = LibriItems(X, Y, context=args['context'])
        test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

        for data, true_y in test_loader:
            data = data.float().to(device)
            
            output = model(data)
            pred_y = torch.argmax(output, axis=1)

            pred_y_list.extend(pred_y.tolist())

In [ ]:
len(pred_y_list) #1943253

In [ ]:
test_submission = my_csv_reader("/content/hw1p2_student_data/test-clean/sample_submission.csv",'all')
print(test_submission[0],test_submission[-1], len(test_submission)) #['id', 'label'] ['1948492', '0'] 1948494

test_submission = test_submission[1:(len(pred_y_list)+1)]
for i,sub in enumerate(test_submission):
    test_submission[i][0] = int(test_submission[i][0]) 
    test_submission[i][1] = pred_y_list[i]

#test_submission is an array of size (1948253 by 2) all real values
output_file_name = 'my_submission.csv'
np.savetxt(output_file_name,test_submission, delimiter=',',fmt='%s',header='id,label',comments='')